# Solving FizzBuzz with Keras

## Overview
- This notebook provides an example Keras network to solve FizzBuzz.
- This is a supervised classification problem, so we'll be 
<img src="https://camo.githubusercontent.com/fedd5d66bea57a430635498de58dc7c6f064f280/68747470733a2f2f64707a6268796262327064636a2e636c6f756466726f6e742e6e65742f63686f6c6c65742f466967757265732f303166696730322e6a7067">

## What is FizzBuzz?
FizzBuzz is a common programming interview problem. Here's the setup.
- Write a program that outputs all the integers from 1 to 100.
- If the number is a multpile of 3, then output the string "Fizz"
- If the number is a multiple of 5, then output the string "Buzz"
- If the number is a multiple of 15, output the string "FizzBuzz"
- For any other number, output that, number itself.

In [1]:
# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
# Specify the number of binary digits
NUM_DIGITS = 10

In [3]:
## Setup the training data for 101-1024. 1024 is the highest number countable with 10 binary digits
raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [4]:
## This is a numpy array of integers.
raw_training_data[1:20]

array([102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 119, 120])

In [13]:
# We need to binary encode our inputs
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [6]:
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]
x_train = np.array(encoded_training_data)

In [7]:
print(binary_encode(1, NUM_DIGITS))
print(binary_encode(2, NUM_DIGITS))
print(binary_encode(3, NUM_DIGITS))

[1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0]


In [8]:
# Now we'll need to one hot encode the training data for y
def fizz_buzz_encode(i):
    if i % 15 == 0:
        return np.array([0, 0, 0, 1]) # encoding for "fizzbuzz"
    elif i % 5 == 0: 
        return np.array([0, 0, 1, 0]) # encoding for "buzz"
    elif i % 3  == 0: 
        return np.array([0, 1, 0, 0]) # encoding for "fizz"
    else:
        return np.array([1, 0, 0, 0]) # encoding for the number output

In [9]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

In [10]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()
model.add(Dense(1000, input_dim=10, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])
model.fit(x_train, y_train, nb_epoch=100, batch_size=128)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys




Epoch 1/100





923/923 [==============================] - 1s 2ms/step - loss: 1.7552 - acc: 0.4334
Epoch 2/100
923/923 [==============================] - 0s 229us/step - loss: 1.1533 - acc: 0.5341
Epoch 3/100
923/923 [==============================] - 0s 195us/step - loss: 1.1387 - acc: 0.5341
Epoch 4/100
923/923 [==============================] - 0s 228us/step - loss: 1.1325 - acc: 0.5341
Epoch 5/100
923/923 [==============================] - 0s 264us/step - loss: 1.1268 - acc: 0.5341
Epoch 6/100
923/923 [==============================] - 0s 217us/step - loss: 1.1177 - acc: 0.5352
Epoch 7/100
923/923 [==============================] - 0s 228us/step - loss: 1.1073 - acc: 0.5352
Epoch 8/100
923/923 [==============================] - 0s 211us/step - loss: 1.0919 - acc: 0.5374
Epoch 9/100
923/923 [==============================] - 0s 220us/step - loss: 1.0787 - acc: 0.5352
Epoch 10/100
923/923 [==============================] - 0s 217us/step - loss: 1.0623 - acc: 0.5395
Epoch 11/100
9

923/923 [==============================] - 0s 314us/step - loss: 0.0516 - acc: 0.9978
Epoch 70/100
923/923 [==============================] - 0s 309us/step - loss: 0.0492 - acc: 0.9989
Epoch 71/100
923/923 [==============================] - 0s 354us/step - loss: 0.0476 - acc: 0.9989
Epoch 72/100
923/923 [==============================] - 0s 397us/step - loss: 0.0471 - acc: 0.9989
Epoch 73/100
923/923 [==============================] - 0s 366us/step - loss: 0.0448 - acc: 0.9989
Epoch 74/100
923/923 [==============================] - 0s 317us/step - loss: 0.0433 - acc: 0.9989
Epoch 75/100
923/923 [==============================] - 0s 260us/step - loss: 0.0430 - acc: 0.9978
Epoch 76/100
923/923 [==============================] - 0s 307us/step - loss: 0.0414 - acc: 0.9989
Epoch 77/100
923/923 [==============================] - 0s 272us/step - loss: 0.0406 - acc: 0.9989
Epoch 78/100
923/923 [==============================] - 0s 258us/step - loss: 0.0387 - acc: 0.9989
Epoch 79/100
923/923 [=

In [11]:
# fizzbuzz to binary
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


In [14]:
# y_train fizzbuzz for prime numbers from 1 to 100
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)
output = np.vectorize(fizz_buzz)(numbers, y_test)
print (output)


['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' '10' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' '20' 'fizz' '22' '23' 'fizz' 'buzz'
 '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz' 'fizz' '37'
 '38' 'fizz' '40' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47' 'fizz' '49'
 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59' 'fizzbuzz'
 '61' '62' 'fizz' '64' '65' 'fizz' '67' '68' '69' 'buzz' '71' 'fizz' '73'
 '74' 'fizzbuzz' '76' '77' 'fizz' '79' '80' 'fizz' '82' '83' 'fizz' 'buzz'
 '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94' 'buzz' 'fizz' '97'
 '98' 'fizz' '100']


In [15]:
# correct answers for fizzbuzz on 1-100

answer = np.array([])
for i in numbers:
    if i % 15 == 0: answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: answer = np.append(answer, "buzz")
    elif i % 3 == 0: answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
print (answer)

['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 '14' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' 'fizz' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' '46' '47'
 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' 'buzz' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' 'fizz' 'buzz'
 '71' 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' 'fizz' '82'
 '83' 'fizz' 'buzz' '86' 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94'
 'buzz' 'fizz' '97' '98' 'fizz' 'buzz']


In [16]:
# Let's evaluate the model's predictions
evaluate = np.array(answer == output)
print (np.count_nonzero(evaluate == True) / 100)

0.93


In [17]:
answer == output

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False])

# Next steps 
- Crank up the amount of training data to improve accuracy
- Solve new problems!